In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv')

In [ ]:
train

Artist  ...                                             Lyrics
0              12 stones  ...  It starts with pain, followed by hate\nFueled ...
1              12 stones  ...  Freedom!\nAlone again again alone\nPatiently w...
2              12 stones  ...  Biting the hand that feeds you, lying to the v...
3              12 stones  ...  You say you know just who I am\nBut you can't ...
4              12 stones  ...  My heart is beating faster can't control these...
...                  ...  ...                                                ...
290178      bobby womack  ...  I'm the best friend he's got I'd give him the ...
290179     bad boys blue  ...  Bad Boys Blue "I Totally Miss You" I did you w...
290180       celine dion  ...  Forgive me for the things That I never said to...
290181          dan bern  ...  The day they found a cure for AIDS The day the...
290182  crawdad republic  ...  Fourth of July has come, it's custom that we g...

[290183 rows x 5 columns]

In [ ]:
test

Song  ...  Track_id
0            craftsmanship  ...      8294
1              come-on-out  ...     21621
2                     riot  ...      3301
3     that-s-what-girls-do  ...      2773
4      believe-in-a-dollar  ...     16797
...                    ...  ...       ...
7930   too-little-too-late  ...     23453
7931             berserker  ...      2724
7932   natural-born-killaz  ...     24147
7933            wide-awake  ...      4150
7934             god-knows  ...     16992

[7935 rows x 6 columns]

# Part 1 -- Genre Classification

## Best Model Using embedding training on-the-fly

### Data Preparation

In [ ]:
en_train = train[train['Language']=='en']

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

remove_punc = re.compile('[^0-9a-z ]') # removing any symbol that is NOT number, letter, space.
STOPWORDS = set(stopwords.words('english')) # only dealing with English Lyrics, so set stopwords to 'english'

def clean_text(text):
   
    text = text.lower() # lowercase text
    text = remove_punc.sub(' ', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with space. 
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwords from text
    return text

en_train['Lyrics'] = en_train['Lyrics'].apply(clean_text)
en_train['Lyrics'] = en_train['Lyrics'].str.replace('\d+', '') # replacing one or more digits by nothing
test['Lyrics'] = test['Lyrics'].apply(clean_text)
test['Lyrics'] = test['Lyrics'].str.replace('\d+', '') # replacing one or more digits by nothing

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
en_train['Lyrics']

0         starts pain followed hate fueled endless quest...
1         freedom alone alone patiently waiting phone ho...
2         biting hand feeds lying voice inside reach beg...
3         say know imagine waits across line thought sti...
4         heart beating faster control feelings anymore ...
                                ...                        
290178    best friend got give shirt back knows trust li...
290179    bad boys blue totally miss wrong foolish heart...
290180    forgive things never said forgive knowing righ...
290181    day found cure aids day found cure aids everyb...
290182    fourth july come custom go make way  place cal...
Name: Lyrics, Length: 250197, dtype: object

In [ ]:
test['Lyrics']

0       folks spend days daydreaming finding clues who...
1       take cold hands put face sharpen axe criminal ...
2       ready time war break fucking doors smash windo...
3       ask change color hair yeah ask need thirty two...
4       believe magic young girl heart music free when...
                              ...                        
7930    tuesday night   hear voice telephone doin fine...
7931    elite forces cloaked fur un sensitive pain bur...
7932    dr dre journey mind maniac doomed killer since...
7933    look hurricane right eye  people dead left die...
7934    deal truth till find proof feel weight step to...
Name: Lyrics, Length: 7935, dtype: object

In [ ]:
en_train['Lyrics'].replace('',np.nan,inplace=True)
en_train.dropna(subset=['Lyrics'],inplace=True)

lyrics = en_train['Lyrics'].values
genre = en_train['Genre'].values

from sklearn.model_selection import train_test_split

lyrics_train, lyrics_val, genre_train, genre_val = train_test_split(lyrics, genre, test_size=0.2, random_state=100)


/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(genre_train)
y_train = encoder.transform(genre_train)
y_val = encoder.transform(genre_val)

In [ ]:
lyrics_test = test['Lyrics'].values
genre_test = test['Genre'].values

y_test = encoder.transform(genre_test)

(unique, counts) = np.unique(y_test, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[   0  810]
 [   1  660]
 [   2  495]
 [   3  960]
 [   4  510]
 [   5  660]
 [   6  810]
 [   7 1110]
 [   8  510]
 [   9 1410]]


In [ ]:
(unique, counts) = np.unique(y_train, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[    0  1521]
 [    1  1615]
 [    2  6525]
 [    3  1819]
 [    4  5781]
 [    5 10702]
 [    6 15256]
 [    7 68956]
 [    8  2207]
 [    9 85774]]


In [ ]:
(unique, counts) = np.unique(y_val, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[    0   369]
 [    1   390]
 [    2  1644]
 [    3   419]
 [    4  1459]
 [    5  2612]
 [    6  3877]
 [    7 17341]
 [    8   557]
 [    9 21371]]


In [ ]:
from keras.utils import np_utils

dummy_y_train = np_utils.to_categorical(y_train)
dummy_y_val = np_utils.to_categorical(y_val)
dummy_y_test = np_utils.to_categorical(y_test)

dummy_y_train.shape, dummy_y_val.shape, dummy_y_test.shape

((200156, 10), (50039, 10), (7935, 10))

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer = Tokenizer(num_words=5000)   # num_words is the size of the vocabulary (top 5000 frequent words in the vocabulary)
tokenizer.fit_on_texts(lyrics_train)    # Updates internal vocabulary based on a list of texts

X_train = tokenizer.texts_to_sequences(lyrics_train)  # Transforms each text in texts to a sequence of integers with its corresponding integer value from the word_index dictionary
X_val = tokenizer.texts_to_sequences(lyrics_val)
X_test = tokenizer.texts_to_sequences(lyrics_test)

# The number of unique words in the whole training text
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index for padding


In [ ]:
def FindMaxLength(lst):
    maxList = max((x) for x in lst)
    maxLength = max(len(x) for x in lst )
  
    return maxList, maxLength

print(FindMaxLength(X_train))

([4999, 1101, 455, 455, 6, 362, 930, 5, 5, 42, 1604, 516, 8, 46, 7, 12, 42, 1131, 72, 1188, 175, 227, 529, 198, 263, 27, 1130, 2976, 2976, 66, 425, 2562, 2562, 12, 89, 1059, 62, 484, 98, 87, 583, 642, 7, 852, 30, 27, 22, 8, 42, 2312, 10, 295, 474, 294, 278, 5, 19, 211, 1101, 1101, 1101, 1101, 1101, 22, 8, 42, 2312, 10, 295, 474, 294, 278, 5, 19, 211, 2976, 2976, 66, 425, 2562, 2562, 12, 89, 1059, 62, 484, 34], 1473)


In [ ]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 500

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_val = pad_sequences(X_val, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

X_train.shape, X_val.shape, X_test.shape

((200156, 500), (50039, 500), (7935, 500))

### Testing the best model
Best Model is CNN model with 4 epoch training

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1-9LYc6KNeX5Yehjk2I4vURPLm6Z_2hoK', dest_path='./model/CNN_model_epoch4.h5')

best_model_otf = tf.keras.models.load_model('./model/CNN_model_epoch4.h5')

best_model_otf.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 50)           250000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 498, 128)          19328     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 200)               25800     
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2010      
Total params: 297,138
Trainable params: 297,138
Non-trainable params: 0
________________________________________________

In [ ]:
loss, accuracy = best_model_otf.evaluate(X_train, dummy_y_train, verbose=False)
print("Training Dataset Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = best_model_otf.evaluate(X_val, dummy_y_val, verbose=False)
print("Validation Dataset Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = best_model_otf.evaluate(X_test, dummy_y_test, verbose=False)
print("Testing Dataset Accuracy:  {:.4f}".format(accuracy))

Training Dataset Accuracy:  0.7406
Validation Dataset Accuracy:  0.6340
Testing Dataset Accuracy:  0.2891


Check the result on the Validation Data

In [ ]:
prediction = best_model_otf.predict(X_val)

labels = ['Country', 'Electric', 'Folk', 'Hip-hop', 'Indie', 'Jazz', 'Metal', 'Pop', 'R&B', 'Rock']
print(prediction)
pred_str = []
for i in range(50039):
  pred_str.append(labels[np.argmax(prediction[i])])

(unique, counts) = np.unique(pred_str, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[3.6733807e-04 7.9748500e-03 4.5280932e-03 ... 8.0110095e-02
  1.0576328e-03 3.9466736e-01]
 [6.9918839e-04 1.1600439e-02 5.7035838e-03 ... 3.1346372e-01
  2.0219320e-03 4.8771316e-01]
 [2.4672030e-02 2.5104450e-03 9.1942489e-02 ... 9.6099094e-02
  4.1738544e-03 7.2039658e-01]
 ...
 [6.2201274e-05 3.3156143e-03 2.7090317e-04 ... 9.6435934e-01
  1.0086500e-02 4.9855388e-03]
 [1.5521170e-02 3.0508093e-03 1.5095961e-01 ... 2.2138397e-01
  5.2005504e-03 5.1328796e-01]
 [4.7118618e-05 6.8530124e-03 1.9875031e-03 ... 4.2481359e-02
  1.1549167e-03 2.7756462e-02]]
[['Folk' '199']
 ['Hip-hop' '21']
 ['Indie' '14']
 ['Jazz' '2445']
 ['Metal' '3625']
 ['Pop' '17590']
 ['Rock' '26145']]


Check the result on the Testing Data

In [ ]:
prediction = best_model_otf.predict(X_test)

labels = ['Country', 'Electric', 'Folk', 'Hip-hop', 'Indie', 'Jazz', 'Metal', 'Pop', 'R&B', 'Rock']
print(prediction)
pred_str = []
for i in range(7935):
  pred_str.append(labels[np.argmax(prediction[i])])

(unique, counts) = np.unique(pred_str, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[7.65836099e-03 5.51757077e-03 6.14404120e-02 ... 4.95431185e-01
  1.51803158e-02 3.09573054e-01]
 [1.22261583e-03 4.29221289e-03 1.29306260e-02 ... 9.87532064e-02
  1.63631234e-03 6.47214830e-01]
 [2.20787861e-05 4.12383210e-03 2.49847560e-03 ... 2.14751959e-02
  3.43735592e-04 4.13438790e-02]
 ...
 [3.43920183e-05 6.48433901e-03 1.70925388e-03 ... 4.30818588e-01
  3.84547631e-03 4.06638207e-03]
 [3.49153561e-04 5.87891182e-03 1.03386985e-02 ... 5.99887483e-02
  3.82853003e-04 6.19561613e-01]
 [7.40548270e-03 3.19059379e-02 4.21678722e-02 ... 1.75882638e-01
  4.84450068e-03 4.31513339e-01]]
[['Folk' '67']
 ['Hip-hop' '16']
 ['Indie' '2']
 ['Jazz' '340']
 ['Metal' '832']
 ['Pop' '2704']
 ['Rock' '3974']]


## Best Model Using Pre-trained Embedding

### Data Preparation

In [ ]:
lyrics = en_train['Lyrics'].values
genre = en_train['Genre'].values
lyrics_test= test['Lyrics'].values
genre_test= test['Genre'].values

from sklearn.model_selection import train_test_split

lyrics_train, lyrics_val, genre_train, genre_val = train_test_split(lyrics, genre, test_size=0.2, random_state=142)

lyrics_train.shape, lyrics_val.shape, genre_train.shape, genre_val.shape 

((200157,), (50040,), (200157,), (50040,))

Create a vocabulary index

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

vectorizer = TextVectorization(max_tokens=5000, output_sequence_length=400)
text_ds = tf.data.Dataset.from_tensor_slices(lyrics_train).batch(128)
vectorizer.adapt(text_ds)

retrieve the computed vocabulary used via vectorizer.get_vocabulary(). Let's print the top 7 words:

In [ ]:
vectorizer.get_vocabulary()[:7]

['', '[UNK]', 'the', 'you', 'i', 'to', 'and']

In [ ]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

Encoding the Genre Column using LabelEncoder

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(genre_train)
y_train = encoder.transform(genre_train)
y_val = encoder.transform(genre_val)
y_test = encoder.transform(genre_test)

In [ ]:
y_train.shape, y_train

((200157,), array([5, 7, 9, ..., 9, 9, 7]))

In [ ]:
(unique, counts) = np.unique(y_test, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[   0  810]
 [   1  660]
 [   2  495]
 [   3  960]
 [   4  510]
 [   5  660]
 [   6  810]
 [   7 1110]
 [   8  510]
 [   9 1410]]


Vectorization for the Lyrics

In [ ]:
X_train = vectorizer(lyrics_train)  # Transforms each text in texts to a sequence of integers with its corresponding integer value from the word_index dictionary
X_val = vectorizer(lyrics_val)


In [ ]:
X_test = vectorizer(lyrics_test)

In [ ]:
print(X_train[0])

tf.Tensor(
[ 100   85  100   35   17    2  199   63   46 1111  216   11 3510  594
   45   11  341   28    4   25   15   78  224   13 1308   34   15   69
   16  431   69   16  431   11 2787    4  123  100   85    6   76  766
   76   10  341  529  114  199   41  223 2116    1   16    7  220   28
    4   22   65   10   65   10   41    8   45   66   32  403 1524  104
   10  619 1214  122   89   91    7 3961    7    1    7    1    7 4580
    7 4923    6    7  539   89   91   31    6   45    6  124    6   43
    6    4   25   47  174  421   48    4  135  240 2100   16    9  188
    4  630  240   31    6   41   69   11    2 1117  100   85    4   88
  144    4   56 1108   10    4  231   12    1   50   28    9   93   30
   84   78  665   10    6   42    4  272  107   10   53  613   47  835
  172  192  637   81   16    7  233  979    6  113  192  311   89   91
    7 3961    7    1    7    1    7 4580    7 4923    6    7  539   89
   91   31    6   45    6  124    6   43    6    4   25   47  174 

In [ ]:
def FindMaxLength(lst):
    
    maxLength = max(len(x) for x in lst )
  
    return maxLength

print(FindMaxLength(X_train))

400


Load pre-trained word embeddings

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2021-05-06 18:40:35--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-05-06 18:40:36--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-05-06 18:40:36--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

dict mapping words (strings) to their NumPy vector representation:

In [ ]:
import os
import pathlib

embeddings_index = {}
f = open('glove.6B.200d.txt', encoding='utf-8')
for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [ ]:
num_tokens = len(voc) + 1
embedding_dim = 200
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 4882 words (118 misses)


In [ ]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

### **Testing the best model**

Best model is single layer LSTM with epoch 9

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1766Zu75bDRb2CiJCuUR1nsGUTGtbnP0j', dest_path='./model/model_SingleL.h5')

best_model_pret = tf.keras.models.load_model('./model/model_SingleL.h5')

best_model_pret.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 400, 200)          1000000   
_________________________________________________________________
lstm_8 (LSTM)                (None, 200)               320800    
_________________________________________________________________
dense_10 (Dense)             (None, 200)               40200     
_________________________________________________________________
dropout_5 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                2010      
Total params: 1,363,010
Trainable params: 1,363,010
Non-trainable params: 0
_________________________________________________________________


In [ ]:
loss, accuracy = best_model_pret.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = best_model_pret.evaluate(X_val, y_val, verbose=False)
print("validation Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.6666
validation Accuracy:  0.6260


In [ ]:
loss, accuracy = best_model_pret.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Testing Accuracy: 0.2858


In [ ]:
prediction = best_model_pret.predict(X_val)

labels = ['Country', 'Electric', 'Folk', 'Hip-hop', 'Indie', 'Jazz', 'Metal', 'Pop', 'R&B', 'Rock']
print(prediction)
pred_str = []
for i in range(50039):
  pred_str.append(labels[np.argmax(prediction[i])])

(unique, counts) = np.unique(pred_str, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[0.0512111  0.00168165 0.10268727 ... 0.08583675 0.00912838 0.65284014]
 [0.00343351 0.01854887 0.04195912 ... 0.2574531  0.0162961  0.45860615]
 [0.05285006 0.00260134 0.22336395 ... 0.08038729 0.00685644 0.54957646]
 ...
 [0.01609807 0.00862634 0.04596556 ... 0.23237742 0.01086019 0.5774801 ]
 [0.00533066 0.02174569 0.02513797 ... 0.35325867 0.01696336 0.43985423]
 [0.00989971 0.01075727 0.03865423 ... 0.1647999  0.00960178 0.6488917 ]]
[['Folk' '25']
 ['Hip-hop' '18']
 ['Jazz' '2302']
 ['Metal' '3549']
 ['Pop' '12707']
 ['Rock' '31438']]


In [ ]:
prediction = best_model_pret.predict(X_test)

labels = ['Country', 'Electric', 'Folk', 'Hip-hop', 'Indie', 'Jazz', 'Metal', 'Pop', 'R&B', 'Rock']
print(prediction)
pred_str = []
for i in range(2000):
  pred_str.append(labels[np.argmax(prediction[i])])

(unique, counts) = np.unique(pred_str, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[8.5345044e-04 2.3709906e-04 2.2864187e-02 ... 1.2144128e-01
  2.0890501e-03 8.3199215e-01]
 [1.5420245e-04 1.1312389e-02 6.6888952e-03 ... 3.5181698e-01
  4.3151695e-03 4.5257539e-01]
 [3.8064285e-05 6.9802208e-03 3.0849972e-03 ... 2.1472165e-01
  2.7510722e-03 5.7729048e-01]
 ...
 [2.4561009e-09 5.9412228e-05 6.1837360e-05 ... 3.6882529e-01
  2.5487245e-05 8.6820006e-02]
 [2.7716463e-04 5.4966202e-03 1.2493230e-02 ... 1.2851885e-01
  2.6247189e-03 7.1468717e-01]
 [7.0836756e-04 1.7914865e-02 1.6546553e-02 ... 1.6856678e-01
  6.4815311e-03 5.7805604e-01]]
[['Folk' '1']
 ['Hip-hop' '1']
 ['Jazz' '82']
 ['Metal' '207']
 ['Pop' '486']
 ['Rock' '1223']]


# Part 2 -- Transfer Learning (Artist Classification)

## Best Model Using Embedding trained on-the-fly

### Data Preparation

In [ ]:
vertical_stack = pd.concat([train, test], axis=0)

In [ ]:
vertical_stack

Artist                     Song  ... Song year Track_id
0           12 stones            world so cold  ...       NaN      NaN
1           12 stones                   broken  ...       NaN      NaN
2           12 stones             3 leaf loser  ...       NaN      NaN
3           12 stones  anthem for the underdog  ...       NaN      NaN
4           12 stones               adrenaline  ...       NaN      NaN
...               ...                      ...  ...       ...      ...
7930  amanda-marshall      too-little-too-late  ...    2006.0  23453.0
7931  aurora-borealis                berserker  ...    2007.0   2724.0
7932           dr-dre      natural-born-killaz  ...    2010.0  24147.0
7933    chris-cornell               wide-awake  ...    2011.0   4150.0
7934        the-coral                god-knows  ...    2007.0  16992.0

[298118 rows x 7 columns]

In [ ]:
vertical_stack['Artist'].value_counts().head(10)

elvis presley         1611
chris brown           1239
elvis costello         923
ella fitzgerald        874
the rolling stones     820
bee gees               811
glee                   765
bad religion           752
beyonce                752
elton john             728
Name: Artist, dtype: int64

In [ ]:
artists = ['elvis presley','chris brown','elvis costello','ella fitzgerald','the rolling stones', 'bee gees', 'glee', 'beyonce', 'bad religion', 'elton john']
top10 = vertical_stack[vertical_stack['Artist'].isin(artists)]
top10

Artist                     Song  ... Song year Track_id
3791       bad religion           american jesus  ...       NaN      NaN
3792       bad religion                 infected  ...       NaN      NaN
3793       bad religion                   a walk  ...       NaN      NaN
3794       bad religion                   sorrow  ...       NaN      NaN
3795       bad religion                      you  ...       NaN      NaN
...                 ...                      ...  ...       ...      ...
289878  ella fitzgerald  too marvelous for words  ...       NaN      NaN
289948      chris brown             i get around  ...       NaN      NaN
290027  ella fitzgerald      ev rything i ve got  ...       NaN      NaN
1574            beyonce                   6-inch  ...    2016.0   5220.0
7431            beyonce             sweet-dreams  ...    2012.0  13805.0

[9275 rows x 7 columns]

In [ ]:
top10['Lyrics'] = top10['Lyrics'].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

remove_punc = re.compile('[^0-9a-z ]') # removing any symbol that is NOT number, letter, space.
STOPWORDS = set(stopwords.words('english')) # only dealing with English Lyrics, so set stopwords to 'english'

def clean_text(text):
   
    text = text.lower() # lowercase text
    text = remove_punc.sub(' ', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with space. 
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwords from text
    return text

top10['Lyrics'] = top10['Lyrics'].apply(clean_text)
top10['Lyrics'] = top10['Lyrics'].str.replace('\d+', '') # replacing one or more digits by nothing

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
top10

Artist                     Song  ... Song year Track_id
3791       bad religion           american jesus  ...       NaN      NaN
3792       bad religion                 infected  ...       NaN      NaN
3793       bad religion                   a walk  ...       NaN      NaN
3794       bad religion                   sorrow  ...       NaN      NaN
3795       bad religion                      you  ...       NaN      NaN
...                 ...                      ...  ...       ...      ...
289878  ella fitzgerald  too marvelous for words  ...       NaN      NaN
289948      chris brown             i get around  ...       NaN      NaN
290027  ella fitzgerald      ev rything i ve got  ...       NaN      NaN
1574            beyonce                   6-inch  ...    2016.0   5220.0
7431            beyonce             sweet-dreams  ...    2012.0  13805.0

[9275 rows x 7 columns]

In [ ]:
lyrics = top10['Lyrics'].values
art = top10['Artist'].values

from sklearn.model_selection import train_test_split

lyrics_train, lyrics_val, art_train, art_val = train_test_split(lyrics, art, test_size=0.1, random_state=100)

In [ ]:
lyrics_train

array(['hahaha ya wanna learn ya abcs abc haha x ya gon like right many ways spell love one way show one way show scare mountain yell til face turn blue come back prove prove would rather let day day actions speak louder say pick letters alphabetbet would say way maybe b something c def initely know want g take hi higher j tryna say love simple abc gotta sing since made clear understand wanna wanna know cool would hurt said know cause p q x ed rest man got eyes u would rather let day day actions speak louder say pick letters alphabetbet would say way maybe b something c def initely know want g take hi higher j tryna say love simple abc',
       'systems life hum accordingly every day every year every century everywhere humans go things get worse worse tell smoke dirty city jungle tranquil hideaway king trees mountains man things sell million dollars pound cut dry good display shelves leave alone animal dangerous beast ones made know feast  lone souls breed losing generation seems kille

In [ ]:
art_train

array(['chris brown', 'bad religion', 'elvis costello', ...,
       'elvis costello', 'elvis presley', 'glee'], dtype=object)

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(art_train)
y_train = encoder.transform(art_train)
y_val = encoder.transform(art_val)

In [ ]:
(unique, counts) = np.unique(art_train, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[['bad religion' 688]
 ['bee gees' 723]
 ['beyonce' 679]
 ['chris brown' 1133]
 ['ella fitzgerald' 784]
 ['elton john' 645]
 ['elvis costello' 818]
 ['elvis presley' 1459]
 ['glee' 677]
 ['the rolling stones' 741]]


In [ ]:
(unique, counts) = np.unique(y_train, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[   0  688]
 [   1  723]
 [   2  679]
 [   3 1133]
 [   4  784]
 [   5  645]
 [   6  818]
 [   7 1459]
 [   8  677]
 [   9  741]]


In [ ]:
(unique, counts) = np.unique(y_val, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[  0  64]
 [  1  88]
 [  2  73]
 [  3 106]
 [  4  90]
 [  5  83]
 [  6 105]
 [  7 152]
 [  8  88]
 [  9  79]]


In [ ]:
from keras.utils import np_utils

dummy_y_train = np_utils.to_categorical(y_train)
dummy_y_val = np_utils.to_categorical(y_val)

dummy_y_train.shape, dummy_y_val.shape

((8347, 10), (928, 10))

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer = Tokenizer(num_words=5000)   # num_words is the size of the vocabulary (top 5000 frequent words in the vocabulary)
tokenizer.fit_on_texts(lyrics_train)    # Updates internal vocabulary based on a list of texts

X_train = tokenizer.texts_to_sequences(lyrics_train)  # Transforms each text in texts to a sequence of integers with its corresponding integer value from the word_index dictionary
X_val = tokenizer.texts_to_sequences(lyrics_val)

# The number of unique words in the whole training text
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index for padding


In [ ]:
vocab_size

24954

In [ ]:
def FindMaxLength(lst):
    maxList = max((x) for x in lst)
    maxLength = max(len(x) for x in lst )
  
    return maxList, maxLength

print(FindMaxLength(X_train))
print(FindMaxLength(X_val))

([4978, 46, 38, 4978, 46, 38, 4978, 46, 38, 1665, 2388, 114, 3, 411, 241, 94, 87, 46, 38, 87, 46, 38, 87, 46, 38, 261, 114, 30, 80, 18, 505, 816, 129, 27, 832, 1179, 299, 1421, 248, 83, 96, 4869, 315, 1047, 50, 792, 83, 309, 309, 309, 309, 309, 171, 1944, 2065, 231, 26, 60, 241, 94, 87, 46, 38, 87, 46, 38, 87, 46, 38, 309, 309, 309, 171, 1944, 2065, 231, 26, 60, 241, 94, 87, 46, 38, 87, 46, 38, 87, 46, 38, 87, 46, 87, 46, 38], 681)
([4702, 438, 117, 402, 1045, 4, 4702, 438, 30, 115, 816, 39, 38, 406, 218, 143, 974, 4, 4702, 438, 141, 30, 9, 4, 9, 2, 2, 29, 4702, 438, 117, 402, 1045, 4, 4702, 438, 141, 30, 115, 4, 4, 4, 4, 4702, 438, 141, 30, 115, 4, 4, 4, 4, 4702, 438, 141, 30, 115], 681)


In [ ]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 500

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_val = pad_sequences(X_val, padding='post', maxlen=maxlen)

X_train.shape, X_val.shape

((8347, 500), (928, 500))

### The best model built from scratch (with on-the-fly embedding)

The best model is CNN trained with 7 epochs

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1UdxZZCljf8Roq6fBFwnMOaaJh1SzZzBb', dest_path='./model/tl_cnn_epoch7.h5')

scratch_otf = tf.keras.models.load_model('./model/tl_cnn_epoch7.h5')

scratch_otf.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 50)           250000    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 498, 128)          19328     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 200)               25800     
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                2010      
Total params: 297,138
Trainable params: 297,138
Non-trainable params: 0
________________________________________________

In [ ]:
loss, accuracy = scratch_otf.evaluate(X_train, dummy_y_train, verbose=False)
print("Training Dataset Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = scratch_otf.evaluate(X_val, dummy_y_val, verbose=False)
print("Validation Dataset Accuracy:  {:.4f}".format(accuracy))

Training Dataset Accuracy:  0.9903
Validation Dataset Accuracy:  0.7909


Check the result on the Training Data

In [ ]:
prediction = scratch_otf.predict(X_train)

labels = ['bad religion', 'bee gees', 'beyonce', 'chris brown', 'ella fitzgerald', 'elton john', 'elvis costello', 'elvis presley', 'glee', 'the rolling stones']
print(prediction)
pred_str = []
for i in range(8347):
  pred_str.append(labels[np.argmax(prediction[i])])


(unique, counts) = np.unique(pred_str, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[1.5320094e-07 6.5559962e-06 4.3202783e-03 ... 2.3106678e-11
  1.3974401e-04 2.0110960e-07]
 [9.9080771e-01 3.7443021e-03 1.9111194e-06 ... 2.7092715e-06
  1.1338397e-03 4.5745386e-05]
 [1.5824487e-03 2.7158063e-02 1.6904833e-02 ... 1.3712190e-02
  4.6609238e-02 2.5653485e-03]
 ...
 [7.9416146e-04 4.0368345e-03 9.9486322e-04 ... 1.3157184e-03
  1.0465756e-02 2.1141142e-04]
 [3.5550676e-09 5.3796880e-03 1.0640523e-04 ... 9.8373377e-01
  2.6470004e-04 1.0302851e-02]
 [4.4256987e-05 1.5027550e-02 1.6775455e-03 ... 2.4237382e-01
  3.3530775e-01 7.3352732e-02]]
[['bad religion' '684']
 ['bee gees' '724']
 ['beyonce' '682']
 ['chris brown' '1134']
 ['ella fitzgerald' '792']
 ['elton john' '651']
 ['elvis costello' '818']
 ['elvis presley' '1464']
 ['glee' '654']
 ['the rolling stones' '744']]


Check the result on the Validation Data

In [ ]:
prediction = scratch_otf.predict(X_val)

labels = ['bad religion', 'bee gees', 'beyonce', 'chris brown', 'ella fitzgerald', 'elton john', 'elvis costello', 'elvis presley', 'glee', 'the rolling stones']
print(prediction)
pred_str = []
for i in range(928):
  pred_str.append(labels[np.argmax(prediction[i])])


(unique, counts) = np.unique(pred_str, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[7.84987561e-11 2.64233861e-08 4.29397733e-06 ... 4.22148244e-14
  5.30774514e-07 2.78395285e-09]
 [1.41349407e-02 1.62381425e-01 1.17883570e-01 ... 1.14015285e-02
  4.11766112e-01 5.36019867e-03]
 [1.32799204e-02 1.71347663e-01 3.35061550e-02 ... 1.63154153e-03
  1.24622166e-01 6.99631113e-04]
 ...
 [1.07996730e-05 9.38685417e-01 1.42218275e-02 ... 7.04260485e-04
  2.48424765e-02 2.18916894e-03]
 [7.01721819e-06 9.95482087e-01 7.30490865e-05 ... 9.01784661e-05
  8.43149377e-04 5.70072676e-04]
 [9.94126201e-01 3.49222799e-04 9.02818374e-06 ... 1.97654663e-06
  3.61830997e-03 6.74436742e-05]]
[['bad religion' '66']
 ['bee gees' '89']
 ['beyonce' '77']
 ['chris brown' '109']
 ['ella fitzgerald' '99']
 ['elton john' '78']
 ['elvis costello' '108']
 ['elvis presley' '154']
 ['glee' '73']
 ['the rolling stones' '75']]


### The best pre-trained model (with on-the-fly embedding) 
the best pre-trained model is CNN


`No Fine Tuning of hyperparameters`

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1-9LYc6KNeX5Yehjk2I4vURPLm6Z_2hoK', dest_path='./model/CNN_model_epoch4.h5')

best_model_otf = tf.keras.models.load_model('./model/CNN_model_epoch4.h5')

best_model_otf.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 50)           250000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 498, 128)          19328     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 200)               25800     
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2010      
Total params: 297,138
Trainable params: 297,138
Non-trainable params: 0
________________________________________________

In [ ]:
loss, accuracy = best_model_otf.evaluate(X_train, dummy_y_train, verbose=False)
print("Training datset Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = best_model_otf.evaluate(X_val, dummy_y_val, verbose=False)
print("Validation dataset Accuracy:  {:.4f}".format(accuracy))

Training datset Accuracy:  0.1324
Validation dataset Accuracy:  0.1207


In [ ]:
prediction = best_model_otf.predict(X_val)

labels = ['bad religion', 'bee gees', 'beyonce', 'chris brown', 'ella fitzgerald', 'elton john', 'elvis costello', 'elvis presley', 'glee', 'the rolling stones']
print(prediction)
pred_str = []
for i in range(928):
  pred_str.append(labels[np.argmax(prediction[i])])

(unique, counts) = np.unique(pred_str, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[1.69143550e-05 7.97159784e-03 4.73974855e-04 ... 7.49000192e-01
  2.93304282e-03 3.02456524e-02]
 [1.88999006e-03 2.07444597e-02 1.62274297e-02 ... 6.19106114e-01
  1.39653692e-02 1.12237409e-01]
 [2.25783698e-03 6.65085111e-03 9.51187965e-03 ... 6.27417505e-01
  4.28851172e-02 2.37856768e-02]
 ...
 [3.95112671e-02 7.02405639e-04 4.89297807e-02 ... 5.60951009e-02
  1.07633760e-02 7.80776143e-01]
 [1.58544618e-03 1.65936947e-02 6.30305987e-03 ... 4.77492034e-01
  5.12489630e-03 3.82561743e-01]
 [1.06670745e-02 8.65931995e-03 7.15162233e-02 ... 3.58107507e-01
  2.43422873e-02 1.17174104e-01]]
[['beyonce' '1']
 ['elton john' '10']
 ['elvis costello' '14']
 ['elvis presley' '558']
 ['the rolling stones' '345']]


`With Fine Tuning of the Hyperparameters`




In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1SJzE0D99TrjshiKmM4ZIaxAtUn1juFC_', dest_path='./model/tl_cnn_retrain_outputlayer.h5')

ft_model_otf = tf.keras.models.load_model('./model/tl_cnn_retrain_outputlayer.h5')

ft_model_otf.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 50)           250000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 498, 128)          19328     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 200)               25800     
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2010      
Total params: 297,138
Trainable params: 2,010
Non-trainable params: 295,128
____________________________________________

In [ ]:
loss, accuracy = ft_model_otf.evaluate(X_train, dummy_y_train, verbose=False)
print("Training datset Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = ft_model_otf.evaluate(X_val, dummy_y_val, verbose=False)
print("Validation dataset Accuracy:  {:.4f}".format(accuracy))

Training datset Accuracy:  0.9909
Validation dataset Accuracy:  0.7909


In [ ]:
prediction = ft_model_otf.predict(X_val)

labels = ['bad religion', 'bee gees', 'beyonce', 'chris brown', 'ella fitzgerald', 'elton john', 'elvis costello', 'elvis presley', 'glee', 'the rolling stones']
print(prediction)
pred_str = []
for i in range(928):
  pred_str.append(labels[np.argmax(prediction[i])])

(unique, counts) = np.unique(pred_str, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[6.58923396e-14 1.23968277e-06 8.85190384e-04 ... 2.28080223e-15
  2.88731215e-04 8.30494074e-09]
 [4.47409292e-07 5.54348886e-01 1.83023240e-05 ... 1.99902263e-02
  4.21032786e-01 5.24050629e-05]
 [1.17936315e-06 1.02256909e-01 7.38651585e-03 ... 4.52942113e-05
  3.14319640e-01 6.33086562e-02]
 ...
 [4.73536730e-08 6.73604012e-01 1.11226555e-05 ... 1.41808335e-02
  3.08149517e-01 2.57797161e-04]
 [1.92605003e-12 9.48007882e-01 5.53565194e-09 ... 4.00142446e-02
  1.88750203e-03 1.33949789e-05]
 [9.12108898e-01 9.29680828e-05 7.16246804e-03 ... 9.27623551e-05
  1.13965175e-03 6.20046444e-03]]
[['bad religion' '66']
 ['bee gees' '114']
 ['beyonce' '72']
 ['chris brown' '106']
 ['ella fitzgerald' '78']
 ['elton john' '74']
 ['elvis costello' '91']
 ['elvis presley' '158']
 ['glee' '84']
 ['the rolling stones' '85']]


## Best Model Using Pre-trained Embedding

### Data Preparation

In [ ]:
top10['Lyrics'] = top10['Lyrics'].astype(str)
lyrics = top10['Lyrics'].values
artist = top10['Artist'].values

In [ ]:
from sklearn.model_selection import train_test_split

lyrics_train, lyrics_val, artist_train, artist_val = train_test_split(lyrics, artist, test_size=0.1, random_state=100)

lyrics_train.shape, lyrics_val.shape, artist_train.shape, artist_val.shape

((8347,), (928,), (8347,), (928,))

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
vectorizer = TextVectorization(max_tokens=5000, output_sequence_length=400)
text_ds = tf.data.Dataset.from_tensor_slices(lyrics_train).batch(128)
vectorizer.adapt(text_ds)

In [ ]:
vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'you', 'i', 'the']

In [ ]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(artist_train)
y_train = encoder.transform(artist_train)
y_val = encoder.transform(artist_val)

In [ ]:
X_train = vectorizer(lyrics_train)  # Transforms each text in texts to a sequence of integers with its corresponding integer value from the word_index dictionary
X_val = vectorizer(lyrics_val)


In [ ]:
import os
import pathlib

embeddings_index = {}
f = open('glove.6B.200d.txt', encoding='utf-8')
for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [ ]:
num_tokens = len(voc) + 1
embedding_dim = 200
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 4823 words (177 misses)


In [ ]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=True,
)

### The Best Model built from Scratch (with pre-trained embedding)

In [ ]:

from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1kUigTOFrNfxGPzQoLsuhN4dXEJa1w5Al', dest_path='./model/modelrnnTF.h5')
model = tf.keras.models.load_model('./model/modelrnnTF.h5')
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 400, 200)          1000000   
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 200)               80200     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               20100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1010      
Total params: 1,101,310
Trainable params: 1,101,310
Non-trainable params: 0
_________________________________________________________________


In [ ]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_val, y_val, verbose=False)
print("validation Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.2152
validation Accuracy:  0.2047


In [ ]:

prediction = model.predict(X_train)

labels = ['bad_religion', 'bee_gees', 'beyonce', 'chris_brown', 'ella_fitzgerald', 'elton_john', 'elvis_costello', 'elvis_presley', 'glee', 'the_rolling_stones']
print(prediction)
pred_str = []
for i in range(2000):
  pred_str.append(labels[np.argmax(prediction[i])])
(unique, counts) = np.unique(pred_str, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[0.05870477 0.13421161 0.04597966 ... 0.23990214 0.10399468 0.09348586]
 [0.05870477 0.13421161 0.04597966 ... 0.23990214 0.10399468 0.09348586]
 [0.05870477 0.13421161 0.04597966 ... 0.23990214 0.10399468 0.09348586]
 ...
 [0.05870477 0.13421163 0.04597966 ... 0.23990214 0.10399466 0.09348585]
 [0.05870477 0.13421161 0.04597966 ... 0.23990211 0.10399466 0.09348587]
 [0.05870477 0.13421161 0.04597966 ... 0.23990214 0.10399468 0.09348586]]
[['beyonce' '196']
 ['chris_brown' '52']
 ['elvis_costello' '1']
 ['elvis_presley' '1751']]


In [ ]:

prediction = model.predict(X_val)

labels = ['bad_religion', 'bee_gees', 'beyonce', 'chris_brown', 'ella_fitzgerald', 'elton_john', 'elvis_costello', 'elvis_presley', 'glee', 'the_rolling_stones']
print(prediction)
pred_str = []
for i in range(700):
  pred_str.append(labels[np.argmax(prediction[i])])
(unique, counts) = np.unique(pred_str, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[0.00622783 0.0077385  0.51898175 ... 0.02311949 0.13300845 0.03021261]
 [0.05870477 0.13421158 0.04597966 ... 0.23990214 0.10399468 0.09348586]
 [0.05870477 0.13421163 0.04597966 ... 0.23990211 0.10399468 0.09348586]
 ...
 [0.05945627 0.13461444 0.0443826  ... 0.24640162 0.09759777 0.09739693]
 [0.05870477 0.13421158 0.04597966 ... 0.23990214 0.10399466 0.09348585]
 [0.05870477 0.13421161 0.04597966 ... 0.23990214 0.10399467 0.09348586]]
[['beyonce' '70']
 ['chris_brown' '15']
 ['elvis_presley' '615']]


### The best pre-trained Model (transfer learning)

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1IJwr8FTxZ2-47pcP239AtSIK88lmQKLX', dest_path='./model/lstmtf.h5')

ft_model = tf.keras.models.load_model('./model/lstmtf.h5')

ft_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 400, 200)          1000000   
_________________________________________________________________
lstm_8 (LSTM)                (None, 200)               320800    
_________________________________________________________________
dense_10 (Dense)             (None, 200)               40200     
_________________________________________________________________
dropout_5 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                2010      
Total params: 1,363,010
Trainable params: 1,363,010
Non-trainable params: 0
_________________________________________________________________


In [ ]:
loss, accuracy = ft_model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = ft_model.evaluate(X_val, y_val, verbose=False)
print("validation Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.7968
validation Accuracy:  0.5991


In [ ]:
prediction = ft_model.predict(X_train)

labels = ['bad religion', 'bee gees', 'beyonce', 'chris brown', 'ella fitzgerald', 'elton john', 'elvis costello', 'elvis presley', 'glee', 'the rolling stones']
print(prediction)
pred_str = []
for i in range(8347):
  pred_str.append(labels[np.argmax(prediction[i])])


(unique, counts) = np.unique(pred_str, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[1.2199609e-08 1.4094957e-03 1.1857445e-03 ... 1.4755252e-03
  1.1122129e-03 8.8469608e-04]
 [9.7155082e-01 6.9270953e-07 7.8308638e-03 ... 9.9372948e-05
  8.9866859e-05 4.3132538e-03]
 [9.4115734e-03 1.9849463e-01 6.2013965e-02 ... 5.2551989e-02
  3.0951244e-01 9.9035040e-02]
 ...
 [2.5030726e-05 9.3926296e-02 6.3269455e-03 ... 3.8149808e-02
  1.3873946e-02 5.3044189e-02]
 [4.0823483e-10 1.3095730e-04 3.4517459e-08 ... 9.9801469e-01
  5.5161794e-04 2.2798097e-04]
 [4.8080565e-05 3.6401615e-01 1.4242223e-03 ... 2.7544230e-01
  1.3833526e-01 1.8551730e-02]]
[['bad religion' '709']
 ['bee gees' '909']
 ['beyonce' '664']
 ['chris brown' '1178']
 ['ella fitzgerald' '824']
 ['elton john' '572']
 ['elvis costello' '747']
 ['elvis presley' '1435']
 ['glee' '839']
 ['the rolling stones' '470']]


In [ ]:

prediction = ft_model.predict(X_val)

labels = ['bad_religion', 'bee_gees', 'beyonce', 'chris_brown', 'ella_fitzgerald', 'elton_john', 'elvis_costello', 'elvis_presley', 'glee', 'the_rolling_stones']
print(prediction)
pred_str = []
for i in range(700):
  pred_str.append(labels[np.argmax(prediction[i])])
(unique, counts) = np.unique(pred_str, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[1.20740323e-19 5.16710156e-12 1.03835891e-10 ... 5.98329570e-05
  1.43649270e-08 2.17969571e-08]
 [1.09899374e-04 3.66530657e-01 2.34725308e-02 ... 5.67903146e-02
  1.18681237e-01 3.22115310e-02]
 [1.59461226e-04 5.11186540e-01 4.82579134e-03 ... 5.74337281e-02
  1.24350727e-01 3.61848772e-02]
 ...
 [4.73456166e-04 2.01499149e-01 2.03059092e-02 ... 3.84292863e-02
  6.05450869e-01 4.98366654e-02]
 [9.19759199e-02 9.77465734e-02 1.22665249e-01 ... 6.51062727e-02
  1.15334108e-01 1.30282968e-01]
 [7.99741685e-01 1.35854702e-04 6.09875135e-02 ... 2.43738946e-03
  2.29082280e-03 3.06914914e-02]]
[['bad_religion' '56']
 ['bee_gees' '84']
 ['beyonce' '57']
 ['chris_brown' '85']
 ['ella_fitzgerald' '79']
 ['elton_john' '51']
 ['elvis_costello' '74']
 ['elvis_presley' '113']
 ['glee' '70']
 ['the_rolling_stones' '31']]


# Part 3 -- Lyrics Generation

## Rock Lyrics Generation

### Data Preparation

In [ ]:
train['Genre'].value_counts()

Rock          121404
Pop           108714
Metal          20291
Jazz           13545
Folk            8644
Indie           8449
R&B             2793
Hip-Hop         2240
Electronic      2213
Country         1890
Name: Genre, dtype: int64

In [ ]:
!pip install -q profanity-check
from profanity_check import predict

In [ ]:
rock = train[train['Genre']=='Rock']
pop = train[train['Genre']=='Pop']
metal = train[train['Genre']=='Metal']

rock.shape, pop.shape, metal.shape

((121404, 5), (108714, 5), (20291, 5))

In [ ]:
df1=pd.Dataframe(rock,columns=['lyrics'])

cleaning offensive words

In [ ]:
count=0
actual_count=0
for i in range(1000):
  #print(type(df1.iloc[i]['Lyrics']))
  if predict([df1.iloc[i]['Lyrics']])==1:
    #print("Offensive word found")
    count=count+1
    try:
      df1=df1.drop([i])
      actual_count=actual_count+1
    except:
      pass
  else:
    pass
print(len(df1))

In [ ]:
rock_sub = df1.sample(n=500)

rock_sub.shape

(500, 5)

In [ ]:
rock_lyrics = rock_sub['Lyrics'].astype('str')

rock_lyrics

49584     (M\nJagger/K\nRichards)\nSometime I wonder why...
101402    Music by Elton John\nLyrics by Bernie Taupin\n...
236804    Here we go again, wasting each other's time\nT...
45705     Avoir, Adieu, Goodnight\nI’m too wrong, to get...
59562     Her voice cut like a razor\nA rose petal with ...
                                ...                        
16347     The wild boys are calling on their way back fr...
1180      I could stay awake just to hear you breathing\...
262853    \noh\none last kiss\noh baby one last kiss\nit...
44165     Don't get me started on politicians\nAnd the l...
21311     Too alarming now to talk about\nTake your pict...
Name: Lyrics, Length: 500, dtype: object

In [ ]:
rocklist = list(rock_lyrics)

rocklist[0]

"(M\nJagger/K\nRichards)\nSometime I wonder why you do these things to me\nSometime I worry girl that you ain't in love with me\nSometime I stay out late, yeah I'm having fun\nYes, I guess you know by now you ain't the only one\nBaby, sweet things that you promised me babe\nSeemed to go up in smoke\nYeah, vanish like a dream\nI wonder why you do these things to me\nCause I'm worried\nI just can't seem to find my way, baby\nOoh, the nights I spent just waiting on the sun\nJust like your burned out cigarette\nYou threw away my love\nWhy did you do that baby\nI wonder why, why you do these things to me\nI'm worried\nLord, I'll find out anyway\nSure going to find myself a girl someday\nTill then I'm worried\nYeah, I just can't seem to find my way\nYeah, I'm a hard working man\nWhen did I ever do you wrong?\nYeah, I get all my money baby\nBring it, bring it all home\nYeah, I'm telling the truth\nSweet things, sweet things that you promised me\nWell I'm worried, I just can't seem to find my 

In [ ]:
import string

def clean_text(txt):
    txt = "".join(i for i in txt if i not in string.punctuation).lower()
    txt = txt.replace("\n"," ")
    return txt 

corpus = [clean_text(x) for x in rocklist]
corpus[0]

'm jaggerk richards sometime i wonder why you do these things to me sometime i worry girl that you aint in love with me sometime i stay out late yeah im having fun yes i guess you know by now you aint the only one baby sweet things that you promised me babe seemed to go up in smoke yeah vanish like a dream i wonder why you do these things to me cause im worried i just cant seem to find my way baby ooh the nights i spent just waiting on the sun just like your burned out cigarette you threw away my love why did you do that baby i wonder why why you do these things to me im worried lord ill find out anyway sure going to find myself a girl someday till then im worried yeah i just cant seem to find my way yeah im a hard working man when did i ever do you wrong yeah i get all my money baby bring it bring it all home yeah im telling the truth sweet things sweet things that you promised me well im worried i just cant seem to find my way baby im worried about you im worried about you tell you s

In [ ]:
len(corpus)

500

In [ ]:
corpus

Input Sequences

`Input sequence is the numerical representation of how our words are arranged` 

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

In [ ]:
inp_sequences, total_words = get_sequence_of_tokens(corpus)


In [ ]:
total_words

7743

Pad Sequences

In [ ]:
from keras.preprocessing.sequence import pad_sequences

max_sequence_len = max([len(x) for x in inp_sequences])
inp_sequences = np.array(pad_sequences(inp_sequences,
                       maxlen = max_sequence_len, padding='pre'))

In [ ]:
print(inp_sequences.shape)
inp_sequences

(94361, 736)


array([[   0,    0,    0, ...,    0,   28,    6],
       [   0,    0,    0, ...,   28,    6, 1385],
       [   0,    0,    0, ...,    6, 1385,  290],
       ...,
       [   0,    0,    0, ...,  161,    6,  358],
       [   0,    0,    0, ...,    6,  358,   16],
       [   0,    0,    0, ...,  358,   16,    7]], dtype=int32)

Predictions and Labels

we will use our input sequence and use the last word of all sequences as labels for all previous words

In [ ]:
predictors, label = inp_sequences[:,:-1],inp_sequences[:,-1]

In [ ]:
import keras.utils as ku

label = ku.to_categorical(label, num_classes=total_words)

In [ ]:
label.shape

(94361, 7743)

### Best NN Language Model for Rock Lyrics Generation

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1X4QnYP7JtHFE1PLxPO-Ci8Gr3ndhiHc2', dest_path='./model/rock01.h5')

rock_generator = tf.keras.models.load_model('./model/rock01.h5')

rock_generator.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 735, 50)           387150    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               60400     
_________________________________________________________________
dense (Dense)                (None, 3871)              390971    
_________________________________________________________________
dense_1 (Dense)              (None, 7743)              29980896  
Total params: 30,819,417
Trainable params: 30,819,417
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def rock_lyrics(seed_text, next_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list],
                     maxlen=max_sequence_len-1,padding='pre')
        
        predicted = rock_generator.predict_classes(token_list, verbose=0)

        output_word = ""
        
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    print(seed_text)

In [ ]:
rock_lyrics("look at", 50)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


look at free me you there ah a thought hand preciso a planks and can i over a that two to years gash making antes spell river beautiful bitter seja cheque be explanation’s nowhere beautiful and is was of showing you here answer oh it um gonna but up a quando this


In [ ]:
rock_lyrics("look at", 100)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


look at free me you there ah a thought hand preciso a planks and can i over a that two to years gash making antes spell river beautiful bitter seja cheque be explanation’s nowhere beautiful and is was of showing you here answer oh it um gonna but up a quando this oh water and hand colar and born my there ah its my there get into a that it chorus it eyes tonight you no call can i que me with you find shes me with you find shes me with you find shes me with you find shes me with


In [ ]:
rock_lyrics("i don't know", 50)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


i don't know clear clear the his you lugar the his and the turning and the rocking and the jukebox of on place friend on changes the drugstore isso of to mommys some in it dogs stirred appear not braved me and in to falam eternity im espacial now put evrything neurotic town


### Statistical Model for Rock Lyrics Generation

In [ ]:
print(ng_model.counts['the'])
print(ng_model.counts['are'])
print(ng_model.counts['love'])
print(ng_model.counts['oh'])

print(ng_model.score('the'))
print(ng_model.score('are'))
print(ng_model.score('love'))
print(ng_model.score('oh'))

print(ng_model.counts[['they']]['are'])
print(ng_model.counts[['are']]['they'])

3613
302
534
369
0.03706857635326466
0.003098452825542742
0.005478721221323922
0.003785857922600238
13
2


In [ ]:
text_list = ng_model.generate(15, random_seed=2)
print(' '.join(word for word in text_list))

you who broke my heart some people do not mess with mister inbetween no dont


In [ ]:
text_list = ng_model.generate(50, random_seed=2)
print(' '.join(word for word in text_list))

you who broke my heart some people do not mess with mister inbetween no dont you worry baby im buying if youre my daughter youre my love ill get it back better late than never i saw you saying that you take my woman where she wants to do it


In [ ]:
text_list = ng_model.generate(50, random_seed=1)
print(' '.join(word for word in text_list))

boy who was hanging his head low more trophies and ideas to follow footprints in the darkest place that cow wrote that im doing it all a big joke whatever it is like porcelain yeah </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>


## Pop Lyrics Generation

In [ ]:
pop_sub = df1.head(500)

pop_sub.shape

(500, 1)

In [ ]:
pop_lyrics = pop_sub['Lyrics'].astype('str')


In [ ]:
poplist = list(pop_lyrics)

poplist[0]

"lives in a little lonely town\nno one's around except for the drinking\nnobody even gets around\nbut those who leave the township sinking\nmay you rot in heaven\ngotta be home by seven\nand the fields burns away\nthe sky breathes it in\nso why sit and wait\nfor the new world to begin\ni'm comin' out across your border\nwith new orders for you to take\ni'd really like to take out yuor daughter\ndown by the water down by the lake\nwhen cold water's on her skin\ni can feel how long it's been\nand the neighbours will all be there\nand no one will know what to wear\nso why sit and wait\nfor the new world to begin\ni got a lot i gotta do\njust to get through the end of the day\nit hardly ever happens\nbut i go to sleep the same anyway\nand you can't believe in yourself\nyou can't believe in anyone else\nso why sit and wait\nfor the new world to begin."

In [ ]:
import string

def clean_text(txt):
    txt = "".join(i for i in txt if i not in string.punctuation).lower()
    txt = txt.replace("\n"," ")
    return txt 


In [ ]:
corpus = [clean_text(x) for x in poplist]
corpus[0]

'lives in a little lonely town no ones around except for the drinking nobody even gets around but those who leave the township sinking may you rot in heaven gotta be home by seven and the fields burns away the sky breathes it in so why sit and wait for the new world to begin im comin out across your border with new orders for you to take id really like to take out yuor daughter down by the water down by the lake when cold waters on her skin i can feel how long its been and the neighbours will all be there and no one will know what to wear so why sit and wait for the new world to begin i got a lot i gotta do just to get through the end of the day it hardly ever happens but i go to sleep the same anyway and you cant believe in yourself you cant believe in anyone else so why sit and wait for the new world to begin'

In [ ]:
len(corpus)

2500

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

In [ ]:
inp_sequences, total_words = get_sequence_of_tokens(corpus)

In [ ]:
total_words

11939

In [ ]:
from keras.preprocessing.sequence import pad_sequences

max_sequence_len = 500
print(max_sequence_len)
inp_sequences = np.array(pad_sequences(inp_sequences,
                       maxlen = max_sequence_len, padding='pre'))

500


In [ ]:
predictors, label = inp_sequences[:,:-1],inp_sequences[:,-1]

In [ ]:
import keras.utils as ku

label = ku.to_categorical(label, num_classes=total_words)

In [ ]:
label.shape

(123751, 6416)

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1VlqLVdrbQ-BRR-K4_1tCvid6sTtZj6EN', dest_path='./model/pop01.h5')

pop_generator = tf.keras.models.load_model('./model/pop01.h5')

pop_generator.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 400, 50)           320800    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               60400     
_________________________________________________________________
dense (Dense)                (None, 3208)              324008    
_________________________________________________________________
dense_1 (Dense)              (None, 6416)              20588944  
Total params: 21,294,152
Trainable params: 21,294,152
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def pop_lyrics(seed_text, next_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list],
                     maxlen=max_sequence_len-1,padding='pre')
        
        predicted = pop_generator.predict_classes(token_list, verbose=0)

        output_word = ""
        
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    print(seed_text)

In [ ]:
print (make_lyrics("They look for", 50))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


They look for me im a tangled puppet and i might be a mess but i sure can survive find myself awake counting sad days 1 2 3 thats too many for too cool for dawn jump a wonderful country but the man hes burning it down yea yea yea yea yea yea
None


In [ ]:
print (make_lyrics("you know well", 50))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


you know well i met you caught up slurring cos shake fire shake machine im a live wire live wire gonna set this town on fire live wire live wire live wire im a live wire live wire im a live wire live wire im a live wire live wire im a live
None


## Metal Lyrics Generation

In [ ]:
metal_sub = df1.sample(n=500)
metal_sub.shape

(500, 1)

In [ ]:
metal_lyrics = metal_sub['Lyrics'].astype('str')


In [ ]:
metallist = list(metal_lyrics)

metallist[0]

'Take a chance n something star you gotta\nAll the live long day\nIf I had to name one thing would data\nBe hard pressed to say\nYou can say that I’m gushy\nAt least you know what I think\nI know that I’m lucky I know you’re on the brink\nYou got it good good good you got it good so good\nYou make up ridiculous words\nWhich mean exactly nothing\nI know just what you mean and that’s the funny thing\nIt’s understood it’s understood\nBuddy buddy buddy I know why you wander\nThrough the sweetness on the ground\nThe sweetness of the flowers\nYou make your way on a cloudy day\nYou’ve got to move\nThe world you prove is too sweet yeah\nWe parted much too soon I know but that’s how it goes\nThere’s someone better out there for you and me\nCan’t wait to get on the road and feel the shows\nAnd be like phenomenon 1 and 2 and 3\nI don’t mean to brag and I don’t mean to boast\nBut I’m the guy could that give you the most\nIt’s good good good it’s understood\nChorus'

In [ ]:
import string

def clean_text(txt):
    txt = txt.lower()
    txt = txt.replace("\n"," ")
    return txt 


In [ ]:
corpus = [clean_text(x) for x in metallist]
corpus[0]

'take a chance n something star you gotta all the live long day if i had to name one thing would data be hard pressed to say you can say that i’m gushy at least you know what i think i know that i’m lucky i know you’re on the brink you got it good good good you got it good so good you make up ridiculous words which mean exactly nothing i know just what you mean and that’s the funny thing it’s understood it’s understood buddy buddy buddy i know why you wander through the sweetness on the ground the sweetness of the flowers you make your way on a cloudy day you’ve got to move the world you prove is too sweet yeah we parted much too soon i know but that’s how it goes there’s someone better out there for you and me can’t wait to get on the road and feel the shows and be like phenomenon 1 and 2 and 3 i don’t mean to brag and i don’t mean to boast but i’m the guy could that give you the most it’s good good good it’s understood chorus'

In [ ]:
len(corpus)

2500

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

In [ ]:
inp_sequences, total_words = get_sequence_of_tokens(corpus)

In [ ]:
from keras.preprocessing.sequence import pad_sequences

max_sequence_len = 500

inp_sequences = np.array(pad_sequences(inp_sequences,
                       maxlen = max_sequence_len, padding='pre'))

In [ ]:
predictors, label = inp_sequences[:,:-1],inp_sequences[:,-1]

In [ ]:
import keras.utils as ku

label = ku.to_categorical(label, num_classes=total_words)

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1FF2m7cQSaxCzlYR1NcRCkUFvdT7Llnun', dest_path='./model/metal01.h5')

metal_generator = tf.keras.models.load_model('./model/metal01.h5')

metal_generator.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 499, 50)           324950    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dense_2 (Dense)              (None, 3249)              328149    
_________________________________________________________________
dense_3 (Dense)              (None, 6499)              21121750  
Total params: 21,835,249
Trainable params: 21,835,249
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def make_lyrics(seed_text, next_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list],
                     maxlen=max_sequence_len-1,padding='pre')
        predicted = metal_generator.predict_classes(token_list, verbose=0)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    print(seed_text)

In [ ]:
print (make_lyrics("light cool", 200))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


light cool the same sweatin in the rain so watch out there's a division in our life how do we take back what's been done what's been said oh my how do we take back 'cuz no one wins the wheel of them what you got so rude boy actin' coy you got nuts liek almond joy i tell ya this is a tribute i'm vexing many mofos but i'm wishing you come a day i didn't sit and mean so much can't hear about it stand up stand up stand up and face up and shakin’ it up now we just go yo dope rhymes might get crazy but they're not already there is not how it has it been since you let me in i need your heart and tears you laugh like then you think that i've done you don't mind yourself and if you dare nothing then waitings what i'll do what you mean screaming silent fears nothing don't try to be a prick to be a a a a a a a a little bit of what i'm a i will not attempt to try to be free but i reserve the right to be stoned i
None


## Statistical model for Lyrics Generation

In [ ]:
!pip install nltk==3.5

import nltk
from nltk.util import ngrams
from nltk.lm import MLE
from nltk import word_tokenize
# we need to download a special component that is used by the tokenizer below -- don't worry about it. 
nltk.download('punkt')

     |████████████████████████████████| 1.4MB 9.2MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp37-none-any.whl size=1434673 sha256=c0084329af7008bbff1db00f3139f25918256a5ecde9a23a8fe91b0cf89e58ff
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')

In [ ]:
en_data = data[data['Language']=='en']
en_data

Artist  ...                                             Lyrics
0              12 stones  ...  It starts with pain, followed by hate\nFueled ...
1              12 stones  ...  Freedom!\nAlone again again alone\nPatiently w...
2              12 stones  ...  Biting the hand that feeds you, lying to the v...
3              12 stones  ...  You say you know just who I am\nBut you can't ...
4              12 stones  ...  My heart is beating faster can't control these...
...                  ...  ...                                                ...
290178      bobby womack  ...  I'm the best friend he's got I'd give him the ...
290179     bad boys blue  ...  Bad Boys Blue "I Totally Miss You" I did you w...
290180       celine dion  ...  Forgive me for the things That I never said to...
290181          dan bern  ...  The day they found a cure for AIDS The day the...
290182  crawdad republic  ...  Fourth of July has come, it's custom that we g...

[250197 rows x 5 columns]

In [ ]:
import re
nltk.download('stopwords')
from nltk.corpus import stopwords

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|\n@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]') # removing any symbol that is NOT number, letter, space, '#', '+', or '_'.
STOPWORDS = set(stopwords.words('english')) # only dealing with English Lyrics, so set stopwords to 'english'

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwords from text
    return text

en_data['Lyrics'] = en_data['Lyrics'].apply(clean_text)
en_data['Lyrics'] = en_data['Lyrics'].str.replace('\d+', '') # replacing one or more digits by nothing

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
lyrics = en_data['Lyrics'].values

taking 30% of total data

In [ ]:
extract = 0.3
lyrics = lyrics[:int(extract*len(lyrics))]

In [ ]:
texts = []
for s in lyrics:
    texts.append(word_tokenize(s))
print(texts[0])

['starts', 'pain', 'followed', 'hate', 'fueled', 'endless', 'questions', 'one', 'answer', 'stain', 'covers', 'heart', 'tears', 'apart', 'like', 'sleeping', 'cancer', 'dont', 'believe', 'men', 'born', 'killers', 'dont', 'believe', 'world', 'saved', 'get', 'start', 'innocent', 'child', 'thorn', 'heart', 'kind', 'world', 'live', 'love', 'divided', 'hate', 'loosing', 'control', 'feelings', 'must', 'dreaming', 'life', 'away', 'world', 'cold', 'sane', 'wheres', 'shame', 'moment', 'time', 'passes', 'can', 'not', 'rewind', 'whos', 'blame', 'start', 'cure', 'sickness', 'heart', 'dont', 'believe', 'men', 'born', 'killers', 'dont', 'believe', 'world', 'cant', 'saved', 'get', 'start', 'innocent', 'child', 'thorn', 'heart', 'kind', 'world', 'live', 'love', 'divided', 'hate', 'selling', 'soul', 'reason', 'must', 'dreaming', 'life', 'away', 'world', 'cold', 'world', 'cold', 'theres', 'sickness', 'inside', 'wants', 'escape', 'feeling', 'get', 'cant', 'find', 'way', 'many', 'times', 'must', 'fall', 'kn

In [ ]:
print(list(ngrams(texts[0], n=1)))
print(list(ngrams(texts[0], n=2)))
print(list(ngrams(texts[0], n=3)))
print(list(ngrams(texts[0], n=4)))
print(list(ngrams(texts[0], n=5)))

[('starts',), ('pain',), ('followed',), ('hate',), ('fueled',), ('endless',), ('questions',), ('one',), ('answer',), ('stain',), ('covers',), ('heart',), ('tears',), ('apart',), ('like',), ('sleeping',), ('cancer',), ('dont',), ('believe',), ('men',), ('born',), ('killers',), ('dont',), ('believe',), ('world',), ('saved',), ('get',), ('start',), ('innocent',), ('child',), ('thorn',), ('heart',), ('kind',), ('world',), ('live',), ('love',), ('divided',), ('hate',), ('loosing',), ('control',), ('feelings',), ('must',), ('dreaming',), ('life',), ('away',), ('world',), ('cold',), ('sane',), ('wheres',), ('shame',), ('moment',), ('time',), ('passes',), ('can',), ('not',), ('rewind',), ('whos',), ('blame',), ('start',), ('cure',), ('sickness',), ('heart',), ('dont',), ('believe',), ('men',), ('born',), ('killers',), ('dont',), ('believe',), ('world',), ('cant',), ('saved',), ('get',), ('start',), ('innocent',), ('child',), ('thorn',), ('heart',), ('kind',), ('world',), ('live',), ('love',), 

In [ ]:
from nltk.lm.preprocessing import padded_everygram_pipeline
train, vocab = padded_everygram_pipeline(4, texts)

In [ ]:
model = MLE(4) 
model.fit(train, vocab)

In [ ]:
print(model.counts['good'])
print(model.counts['new'])
print(model.counts['questions'])

28853
17333
968


In [ ]:
print(model.score('good'))
print(model.score('new'))
print(model.score('question'))

0.002491084614252271
0.001496481115268243
0.0001295920010394988


In [ ]:
print(model.counts[['seas']]['pirate'])

1


In [ ]:
word_list = model.generate(100, random_seed=40)
print(' '.join(word for word in word_list))

ive thinking babe things aint went friends lovers arguing like fight try compromise hard let go wait around youll lose wife play long youll lose life got ta good life could go far high class woman tryn give line left early felt fine cant stay theres something wrong ironic aint thats way life thats thank god girls thank god girls halle jesu christe tennessee la thank god girls reckoning day better bow pray shes big shes strong shes energetic sweaty overalls thank god girls reckoning day better bow pray shes big shes strong shes soft shes honest really helps good
